# Задача о назначениях

Постановка:

$\begin{cases}
\sum\limits_{ij}c_{ij}x_{ij}\to\min\\
\sum\limits_i x_{ij}=1\\
\sum\limits_j x_{ij}=1\\
x_{ij}\in\{0,1\}
\end{cases}$

$x_{ij}$ &mdash; берет ли $i$-й работник $j$-ю работу. $c_{ij}$&mdash; стоимость выполнения $j$-й работы $i$-м работником

In [5]:
from pyomo.environ import *
import pyomo.opt as opt

model = AbstractModel()

# Nodes in the network
model.N = Set()
# Network arcs
model.A = Set(within=model.N*model.N)

# Source node
model.s = Param(within=model.N)
# Sink node
model.t = Param(within=model.N)
# Flow capacity limits
model.c = Param(model.A)

# The flow over each arc
model.f = Var(model.A, within=NonNegativeReals)

# Maximize the flow into the sink nodes
def total_rule(model):
    return sum(model.f[i,j] for (i, j) in model.A if j == value(model.t))
model.total = Objective(rule=total_rule, sense=maximize)

# Enforce an upper limit on the flow across each arc
def limit_rule(model, i, j):
    return model.f[i,j] <= model.c[i, j]
model.limit = Constraint(model.A, rule=limit_rule)

# Enforce flow through each node
def flow_rule(model, k):
    if k == value(model.s) or k == value(model.t):
        return Constraint.Skip
    inFlow  = sum(model.f[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.f[i,j] for (i,j) in model.A if i == k)
    return inFlow == outFlow
model.flow = Constraint(model.N, rule=flow_rule)


In [7]:
results = opt.solve(instance) # Solving a model instance  
instance.load(results) # Loading solution into results object

AttributeError: 'module' object has no attribute 'solve'

In [12]:
# Translated to Pyomo from AMPL model source at:
#   https://projects.coin-or.org/Ipopt/wiki/IpoptAddFeatures
#
# This Pyomo example is formulated as a python script.
# To run this script execute the following command:
#
# $ coopr_python ipopt_example.py
#
# Execution of this script requires that the ipopt
# solver is in the current search path for executables
# on this system. This example was tested using Ipopt
# version 3.10.2

from coopr.pyomo import *
from coopr.opt import SolverFactory

### Create the ipopt solver plugin using the ASL interface
solver = 'ipopt'
solver_io = 'nl'
stream_solver = False    # True prints solver output to screen
keepfiles =     False    # True prints intermediate file names (.nl,.sol,...)
opt = SolverFactory(solver,solver_io=solver_io)
###

### Create the example model
model = ConcreteModel()
model.x1 = Var(bounds=(1,5),initialize=1.0)
model.x2 = Var(bounds=(1,5),initialize=5.0)
model.x3 = Var(bounds=(1,5),initialize=5.0)
model.x4 = Var(bounds=(1,5),initialize=1.0)
model.obj = Objective(expr=model.x1*model.x4*(model.x1+model.x2+model.x3) + model.x3)
model.inequality = Constraint(expr=model.x1*model.x2*model.x3*model.x4 >= 25.0)
model.equality = Constraint(expr=model.x1**2 + model.x2**2 + model.x3**2 + model.x4**2 == 40.0)
###

### Declare all suffixes
# Incoming Ipopt bound multipliers (obtained from solution)
model.ipopt_zL_out = Suffix(direction=Suffix.OUT)
model.ipopt_zU_out = Suffix(direction=Suffix.OUT)
# Outgoing Ipopt bound multipliers (sent to solver)
model.ipopt_zL_in = Suffix(direction=Suffix.IN)
model.ipopt_zU_in = Suffix(direction=Suffix.IN)
###

### Generate the constraint expression trees if necessary
if solver_io != 'nl':
    # only required when not using the ASL interface
    model.preprocess()
###

### Send the model to ipopt and collect the solution
print 
print "INITIAL SOLVE"
results = opt.solve(model,keepFiles=keepfiles,tee=stream_solver)
# load the results (including any values for previously declared
# OUT / INOUT Suffix components)
model.load(results)
###

### Print Solution
print "   %7s %12s %12s" % ("Value","ipopt_zL_out","ipopt_zU_out")
for v in [model.x1,model.x2,model.x3,model.x4]:
    print "%s %7g %12g %12g"%( v,
                               value(v),
                               model.ipopt_zL_out.getValue(v),
                               model.ipopt_zU_out.getValue(v) )
###


### Set Ipopt options for warm-start
# The current values on the ipopt_zU_out and
# ipopt_zL_out suffixes will be used as initial
# conditions for the bound multipliers to solve
# the new problem
for var in [model.x1,model.x2,model.x3,model.x4]:
    model.ipopt_zL_in.setValue(var,model.ipopt_zL_out.getValue(var))
    model.ipopt_zU_in.setValue(var,model.ipopt_zU_out.getValue(var))
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-6
opt.options['warm_start_mult_bound_push'] = 1e-6
opt.options['mu_init'] = 1e-6
###

### Send the model to ipopt and collect the solution
print 
print "WARM-STARTED SOLVE"
results = opt.solve(model,keepFiles=keepfiles,tee=stream_solver)
# load the results (including any values for previously declared
# OUT / INOUT Suffix components)
model.load(results)
###

### Print Solution
print "   %7s %12s %12s" % ("Value","ipopt_zL_out","ipopt_zU_out")
for v in [model.x1,model.x2,model.x3,model.x4]:
    print "%s %7g %12g %12g"%( v,
                               value(v),
                               model.ipopt_zL_out.getValue(v),
                               model.ipopt_zU_out.getValue(v) )
###


ImportError: No module named coopr.pyomo